#Table Content
1. Importing Libraries & Data
2. Security Data Check
3. Regional Segment
4. Customers with low income
5. Exporting Dataframes

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#create pathway to import data

path = r'/Users/office/Desktop/Instacart Basket Analysis'

In [3]:
#import data

ords_prods = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','ords_prods_customers_left.pkl'))
ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,first name,last name,gender,state,age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2019-02-17,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2019-02-17,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2019-02-17,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2019-02-17,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2019-02-17,3,married,40423,both


#02 Checking dataframe for PII data and security

In [5]:
ords_prods.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'mean_spend', 'spender_flag', 'median_order_frequency',
       'order_frequency_flag', 'first name', 'last name', 'gender', 'state',
       'age', 'date_joined', 'n_dependants', 'fam_status', 'income', '_merge'],
      dtype='object')

In [9]:
#dropping columns for PII 'first name' and 'last name'

ords_prods.drop(columns=['first name','last name'], inplace=True)
ords_prods.columns


Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'mean_spend', 'spender_flag', 'median_order_frequency',
       'order_frequency_flag', 'gender', 'state', 'age', 'date_joined',
       'n_dependants', 'fam_status', 'income'],
      dtype='object')

#03. Creating Region column from the state column

In [10]:
#Check states

ords_prods['state'].value_counts(dropna = False)

state
Pennsylvania            667710
California              660447
Rhode Island            657655
Georgia                 657072
New Mexico              655143
Arizona                 654545
North Carolina          652338
Oklahoma                652263
Alaska                  649020
Minnesota               648408
Massachusetts           647000
Wyoming                 644929
Virginia                642075
Missouri                641313
Texas                   641058
Colorado                639752
Maine                   639149
North Dakota            638999
Alabama                 638617
Louisiana               638174
Kansas                  637991
Delaware                637447
South Carolina          637348
Oregon                  636949
Arkansas                636706
New York                636697
Nevada                  636697
Montana                 635835
South Dakota            634340
Illinois                633490
Hawaii                  633445
Washington              633421
Mi

#Create Region

In [11]:
Northeast = ['Maine','New Hampshire','Vermont','Massachusetts','Rhode Island','Connecticut','New York','Pennsylvania','New Jersey']

In [13]:
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']

In [14]:
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']

In [15]:
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [16]:
ords_prods.loc[ords_prods['state'].isin(Northeast), 'region'] = 'Northeast'

In [17]:
ords_prods.loc[ords_prods['state'].isin(Midwest), 'region'] = 'Midwest'

In [18]:
ords_prods.loc[ords_prods['state'].isin(South), 'region'] = 'South'

In [19]:
ords_prods.loc[ords_prods['state'].isin(West), 'region'] = 'West'

In [20]:
ords_prods['region'].value_counts(dropna = False)

region
South        10801610
West          8300445
Midwest       7603810
Northeast     5728347
Name: count, dtype: int64

In [21]:
#Create a crosstab to find out whether there’s a difference in spending habits between the different U.S. regions


region_crosstab = pd.crosstab(ords_prods['spender_flag'], ords_prods['region'], dropna = False)

In [22]:
#check

region_crosstab

region,Midwest,Northeast,South,West
spender_flag,,,,
High spender,156129,108343,210182,160807
Low spender,7447681,5620004,10591428,8139638


In [23]:
region_crosstab.to_clipboard()

#04 Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data

In [30]:
#create exclusion flag for low activity customers

ords_prods.loc[ords_prods['max_order'] <5, 'activity_flag'] =  'low activity'

In [28]:
ords_prods.loc[ords_prods['max_order'] >-5, 'activity_flag'] =  'normal activity'

In [31]:
#checking flag

ords_prods['activity_flag'].value_counts(dropna = False)

activity_flag
normal activity    30992664
low activity        1441548
Name: count, dtype: int64

In [27]:
ords_prods.shape

(32434212, 32)

In [33]:
#create subset of low_activity customers

df_low_activity = ords_prods[ords_prods['activity_flag'] =='low activity']

In [34]:
#create a subset for normal activity customers

df_normal_activity = ords_prods[ords_prods['activity_flag'] =='normal activity']

In [35]:
#Check sum of both subsets

len(df_low_activity) + len(df_normal_activity)

32434212

#05. Exporting Dataframes

In [36]:
#Export normal activity customers dataframe

df_low_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_low_activity.pkl'))